# PySpark - Create Data Frame from API

In [ ]:
# Create Spark Session

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Read data from API") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

spark

In [ ]:
# Create Python function to read data from API
import requests, json

def read_api(url: str):
    normalized_data = dict()
    data = requests.get(api_url).json() 
    normalized_data["_data"] = data # Normalize payload to handle array situtations
    return json.dumps(normalized_data)

In [ ]:
api_url = r"https://api.coindesk.com/v1/bpi/currentprice.json"
# api_url = "https://api.wazirx.com/sapi/v1/tickers/24hr"

# Read data into Data Frame
# Create payload rdd
payload = json.loads(read_api(api_url))
payload_rdd = spark.sparkContext.parallelize([payload])

# Read from JSON
df = spark.read.json(payload_rdd)
df.select("_data").printSchema()

In [ ]:
# Expand root element to read Struct Data
df.select("_data.*").show(truncate=False)

In [ ]:
# Expand further elements to read USD data
df.select("_data.*").select("bpi.*").select("USD.*").show(truncate=False)